In [1]:
from openai import OpenAI
import numpy as np

In [2]:
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_KEY'))

In [6]:
with open('messi.txt', 'r') as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [7]:
CHUNNK_SIZE = 70
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈ', 'mesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine ', 'professional footballer who plays as a forward for and captains both M', 'ajor League Soccer club Inter Miami and the Argentina national team. W', 'idely regarded as one of the greatest players of all time, Messi set n', 'umerous records for individual accolades won throughout his profession', "al footballing career such as eight Ballon d'Or awards and eight times", " being named the world's best player by FIFA.[note 2] He is the most d", 'ecorated player in the history of professional football having won 45 ', 'team trophies,[note 3] including twelve league titles, four UEFA Champ', 'ions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds t', 'he records for most European Golden Shoes (6), most goals for a single', ' club (672, with Barcelona), most goals (474), hat-tricks (36) and ass', 'ists (192) in La Liga, most matches 

In [8]:
embedded_chunks = []
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

[('Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈ', [0.03018535114824772, -0.0581023246049881, -0.047516558319330215, -0.013381442986428738, -0.03080219216644764, 0.018684275448322296, 0.016963092610239983, 0.0040218946523964405, -0.029071060940623283, -0.015560280531644821, -0.07326464354991913, 0.004601425491273403, -0.0026936994399875402, -0.027538910508155823, 0.02586747333407402, 0.05949518829584122, -0.05742579326033592, -0.004549193195998669, 0.0009302341495640576, -0.05141657963395119, 0.036015480756759644, 0.009884359315037727, 0.020375609397888184, 0.009287417866289616, 0.024096546694636345, 0.006367377936840057, -0.04282061755657196, 0.017550086602568626, -0.008088559843599796, 0.01668452098965645, 0.07425954937934875, -0.023041950538754463, 0.013580423779785633, 0.0012125378707423806, -0.00312896934337914, -0.0009190415148623288, -0.031040968373417854, -0.019778668880462646, -0.012704908847808838, -0.032095566391944885, -0.02236541546881199, -0.02883

In [9]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

In [10]:
questions = ["Who is Messi?", "Why is Messi so famous?", "Where has messi played?"]

In [11]:
question = questions[0]

In [12]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[0.012747385539114475, -0.0725535973906517, -0.05252669379115105, 0.013636738061904907, -0.03618896007537842, -0.006598775740712881, 0.00929977186024189, 0.03438829630613327, 0.003911504056304693, -0.01387829054147005, -0.05503005534410477, 0.016996514052152634, -0.010963190346956253, -0.025648485869169235, 0.026790371164679527, 0.03443221375346184, -0.06131042167544365, 0.0072410861030220985, -0.011226702481508255, -0.017765089869499207, -0.0006323598208837211, 0.012121545150876045, -0.003090774407610297, 0.012890120968222618, 0.03970245271921158, 0.0056215859949588776, -0.044116273522377014, -0.012988938018679619, -0.025648485869169235, 0.043435536324977875, 0.05362465977668762, -0.04347945377230644, 0.0504625178873539, -0.011879992671310902, 0.006274875719100237, 0.012857181951403618, -0.01991710439324379, -0.020477065816521645, -0.008778238669037819, -0.0025582609232515097, -0.033224452286958694, -0.04501660540699959, -0.008750788867473602, 0.03702341392636299, -0.00426559802144765

In [13]:
best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

In [14]:
print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)

Relevant information: ational, Messi is the national team's all-time leading goalscorer and 


In [15]:
print(completion.choices[0].message.content)

Lionel Messi is an Argentine professional football (soccer) player widely regarded as one of the greatest players of all time. Born on June 24, 1987, in Rosario, Argentina, Messi has had an illustrious career marked by numerous records and accolades.

He spent the majority of his professional club career at FC Barcelona, where he played from 2004 to 2021. During his time at Barcelona, Messi won numerous La Liga titles, UEFA Champions League titles, and FIFA Club World Cups among other achievements. He is Barcelona's all-time leading goalscorer.

In August 2021, Messi transferred to Paris Saint-Germain (PSG) after Barcelona could not renew his contract due to financial restrictions. At PSG, he continues to play at a high level.

Internationally, Messi is the Argentine national team's all-time leading goalscorer. He has represented Argentina in multiple tournaments, including the Copa América and FIFA World Cup. In 2021, he led Argentina to victory in the Copa América, which was his firs

In [16]:
# for the other questions
question = questions[1]

In [17]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)



[0.039022449404001236, -0.0640222355723381, -0.03862806037068367, 0.04767706245183945, 0.008112330920994282, -0.011448192410171032, -0.0017788518453016877, 0.05942104756832123, 0.015906961634755135, -0.02979816496372223, -0.05990307405591011, 0.01315720472484827, -0.0328875333070755, -0.013913113623857498, 0.009481731802225113, 0.010615596547722816, -0.06266378611326218, -0.00796443596482277, -0.004026039037853479, -0.027453750371932983, -0.0056474097073078156, 0.03275607153773308, 0.003357771085575223, 0.010034970007836819, 0.030170641839504242, 0.014044576324522495, -0.03924155235290527, 0.013836427591741085, -0.0033605098724365234, 0.028111062943935394, 0.027103183791041374, -0.026949811726808548, 0.02548181265592575, -0.028417808935046196, -0.03354484587907791, -0.022206205874681473, -0.0067812735214829445, -0.014625201933085918, 0.02269919030368328, -0.011919265612959862, -0.018470481038093567, -0.043930381536483765, -0.0033112114761024714, 0.029469508677721024, -0.016575228422880

In [18]:
print(completion.choices[0].message.content)

Lionel Messi is widely considered one of the greatest football players of all time, and his fame can be attributed to several factors:

1. **Exceptional Talent and Skill**: Messi is known for his incredible dribbling ability, vision, precision in passing and finishing, and his agility on the field. His technical skills and football intelligence are unmatched.

2. **Record-Breaking Achievements**: Throughout his career, Messi has broken numerous records. He holds records for the most goals scored in a calendar year, the most goals in a European club season, and many others. 

3. **Consistency and Longevity**: Messi has maintained a high level of performance for nearly two decades. His consistency over the years has been remarkable, winning multiple awards and accolades continuously.

4. **Success with FC Barcelona**: Messi spent the majority of his career with Barcelona, where he won numerous La Liga titles, UEFA Champions League trophies, and other cup victories. His success with the c

In [19]:
question = questions[2]

In [20]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)

[-0.036236200481653214, -0.07004107534885406, -0.000845998409204185, 0.005920527968555689, -0.026090065017342567, -0.0164231825619936, 0.020911796018481255, 0.034903645515441895, -0.03546472266316414, 0.01974288560450077, -0.04881367087364197, -0.011338424868881702, -0.010286405682563782, -0.001358857611194253, 0.009415567852556705, 0.030976107344031334, -0.027936942875385284, 0.015067246742546558, -0.004818830173462629, 0.023717178031802177, 0.016598518937826157, 0.04189372435212135, -0.009462323971092701, -0.002127415733411908, 0.013372327201068401, -0.014623060822486877, 0.004465235397219658, 9.442598820896819e-05, 0.00046135406591929495, 0.03756875917315483, 0.0626068040728569, -0.03212163969874382, 0.007837539538741112, 0.019777953624725342, 0.014097051694989204, -0.028755180537700653, 0.01151376124471426, 0.005511409603059292, -0.010730591602623463, -0.0191934984177351, -0.008813579566776752, 0.0024649384431540966, 0.006452381610870361, 0.014319144189357758, -0.002559912391006946

In [21]:
print(completion.choices[0].message.content)

Lionel Messi has had an illustrious career, playing for both club and country. Here's a summary of where he has played:

**For Club:**

1. **FC Barcelona (2000–2021):**
   - Youth Career: Messi joined Barcelona's youth academy, La Masia, in 2000.
   - Senior Career: Messi made his competitive debut for Barcelona's first team in 2004 and played for the club until 2021. During his time there, he became the club's all-time top scorer and won numerous trophies, including 10 La Liga titles and 4 UEFA Champions League titles.

2. **Paris Saint-Germain (PSG) (2021–2023):**
   - Messi signed with PSG in August 2021 after leaving Barcelona due to financial constraints at the club. He played for PSG until 2023.

3. **Inter Miami CF (2023–Present):**
   - In 2023, Messi joined Inter Miami CF, a Major League Soccer (MLS) team based in the United States.

**For Country:**

- **Argentina National Team (2005–Present):**
  - Messi has represented Argentina at various levels, including the U20 and U23 